In [2]:
import pandas as pd
df = pd.read_csv('../vessel_movements_dataset.csv')
df

,vessel_id,vessel_type_new,timestamp,timestamp_epoch,latitude,longitude,speed_knots,in_anchorage,in_port_boundary,safety_score,...,aux_engine_fuel_type,boil_engine_fuel_type,engine_type,mep,vref,sfc_me,sfc_ae,sfc_ab,ael,abl
0,10498900,Chemical/Products Tanker,2025/03/29 19:02:27+00,1743274947,1.331667,104.660000,12.59,NaN,Singapore,4,...,DISTILLATE FUEL,DISTILLATE FUEL,SSD,5050,13.23,156.7,209.1,300,746,125
1,10498900,Chemical/Products Tanker,2025/03/29 20:02:27+00,1743278547,1.280000,104.861667,12.47,NaN,NaN,4,...,DISTILLATE FUEL,DISTILLATE FUEL,SSD,5050,13.23,156.7,209.1,300,746,125
2,10498900,Chemical/Products Tanker,2025/03/29 21:02:28+00,1743282148,1.155000,105.026667,12.55,NaN,NaN,4,...,DISTILLATE FUEL,DISTILLATE FUEL,SSD,5050,13.23,156.7,209.1,300,746,125
3,10498900,Chemical/Products Tanker,2025/03/29 22:02:25+00,1743285745,1.015000,105.180000,12.47,NaN,NaN,4,...,DISTILLATE FUEL,DISTILLATE FUEL,SSD,5050,13.23,156.7,209.1,300,746,125
4,10498900,Chemical/Products Tanker,2025/03/29 23:02:27+00,1743289347,0.865000,105.323333,12.52,NaN,NaN,4,...,DISTILLATE FUEL,DISTILLATE FUEL,SSD,5050,13.23,156.7,209.1,300,746,125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13211,10744390,Chemical/Products Tanker,2025/06/05 18:00:02+00,1749146402,-18.971148,118.200507,12.68,NaN,NaN,4,...,DISTILLATE FUEL,DISTILLATE FUEL,SSD,10320,12.00,185.0,230.0,300,739,2493
13212,10744390,Chemical/Products Tanker,2025/06/05 19:00:11+00,1749150011,-19.173822,118.262568,12.53,NaN,NaN,4,...,DISTILLATE FUEL,DISTILLATE FUEL,SSD,10320,12.00,185.0,230.0,300,739,2493
13213,10744390,Chemical/Products Tanker,2025/06/05 20:00:57+00,1749153657,-19.376667,118.316667,12.62,NaN,NaN,4,...,DISTILLATE FUEL,DISTILLATE FUEL,SSD,10320,12.00,185.0,230.0,300,739,2493
13214,10744390,Chemical/Products Tanker,2025/06/05 21:00:59+00,1749157259,-19.581667,118.363333,10.47,NaN,Port Hedland,4,...,DISTILLATE FUEL,DISTILLATE FUEL,SSD,10320,12.00,185.0,230.0,300,739,2493


In [11]:
df.columns

Index(['vessel_id', 'vessel_type_new', 'timestamp', 'timestamp_epoch',
       'latitude', 'longitude', 'speed_knots', 'in_anchorage',
       'in_port_boundary', 'safety_score', 'dwt', 'fuel_category',
       'main_engine_fuel_type', 'aux_engine_fuel_type',
       'boil_engine_fuel_type', 'engine_type', 'mep', 'vref', 'sfc_me',
       'sfc_ae', 'sfc_ab', 'ael', 'abl '],
      dtype='object')

In [38]:
print(df['main_engine_fuel_type'].unique())
print(df['aux_engine_fuel_type'].unique())
print(df['boil_engine_fuel_type'].unique())

['Methanol' 'DISTILLATE FUEL' 'LNG' 'Ammonia' 'Ethanol' 'LPG (Butane)'
 'Hydrogen' 'LPG (Propane)']
['DISTILLATE FUEL']
['DISTILLATE FUEL']


In [32]:
# df = your vessel movement dataset

# Get unique vessel IDs
df_latest = (
    df.sort_values('timestamp')
      .groupby('vessel_id', as_index=False)
      .tail(1)
)

df_first = (
    df.sort_values('timestamp')
      .groupby('vessel_id', as_index=False)
      .head(1)
)

df_vessel = pd.concat([df_first, df_latest])
id_map = {id_val: i for i, id_val in enumerate(df['vessel_id'].unique())}

# Create a temporary rank column based on the map
df_vessel['id_rank'] = df_vessel['vessel_id'].map(id_map)

# Sort by the rank, then by the second field (e.g., 'Date' or 'Value')
df_vessel = df_vessel.sort_values(by=['id_rank', 'timestamp']).drop(columns='id_rank')

# df_latest.to_csv('../lastVesselInstance.csv', index=False)
df_vessel.to_csv('../vessels_info.csv', index=False)

### Step 1

In [20]:
# step 1
step1_df = df.copy()

def classify_movement(row):
    in_anchorage = (
        0 if pd.isna(row['in_anchorage']) else 1
    )

    in_port_boundary = (
        0 if pd.isna(row['in_port_boundary']) else 1
    )

    speed = row['speed_knots']

    if in_anchorage == 1 and speed < 1:
        return 'Anchorage'
    elif in_port_boundary == 1 and speed > 1:
        return 'Maneuver'
    elif in_port_boundary == 0 and speed >= 1:
        return 'Transit'
    else:
        return 'Drifting'
step1_df['movement_mode'] = step1_df.apply(classify_movement, axis=1)
step1_df.to_csv('../vessel_movements_dataset_modified.csv', index=False)

## Step 2

In [21]:
step2_df = step1_df.copy()
step2_df['timestamp'] = pd.to_datetime(step2_df['timestamp'], format='mixed')
# step2_df = step2_df.sort_values(['vessel_id', 'timestamp'])

step2_df['activity_hours'] = (
    step2_df.groupby('vessel_id')['timestamp']
      .diff()
      .dt.total_seconds()
      .div(3600)
      .fillna(0)
)
step2_df.to_csv('../vessel_movements_dataset_modified.csv', index=False)


## Step 4

In [70]:
import pandas as pd
# 4a
cal_fac_df = pd.read_excel('../calculation_factors.xlsx', sheet_name='Cf')
cal_fac_df


step4_df = pd.read_csv('../vessel_movements_dataset_modified.csv')
cal_fac_df.dropna()
cal_fac_df.at[1, 'Fuel Type'] = 'DISTILLATE FUEL'
step4_df = step4_df.rename(columns={'abl ': 'abl'})


# ['Methanol' 'DISTILLATE FUEL' 'LNG' 'Ammonia' 'Ethanol' 'LPG (Butane)'
#  'Hydrogen' 'LPG (Propane)']
# ['DISTILLATE FUEL']
# ['DISTILLATE FUEL']
lcv_map = (
    cal_fac_df
    .set_index('Fuel Type')['LCV (MJ/kg)']
    .to_dict()
)

lcv_map

step4_df['lcv_me'] = step4_df['main_engine_fuel_type'].map(lcv_map)
step4_df['lcv_ae'] = step4_df['aux_engine_fuel_type'].map(lcv_map)
step4_df['lcv_ab'] = step4_df['boil_engine_fuel_type'].map(lcv_map)

DISTILLATE_LCV = 42.7

step4_df['sfc_adjusted_me'] = step4_df['sfc_me'] * (DISTILLATE_LCV / step4_df['lcv_me'])
step4_df['sfc_adjusted_ae'] = step4_df['sfc_ae'] * (DISTILLATE_LCV / step4_df['lcv_ae'])
step4_df['sfc_adjusted_ab'] = step4_df['sfc_ab'] * (DISTILLATE_LCV / step4_df['lcv_ab'])

step4_df = step4_df.drop(['lcv_me', 'lcv_ae', 'lcv_ab'], axis=1)

# 4b fuel concumption

step4_df['me_fual_consumption(tonnes)'] = (step4_df['load_factor'] * step4_df['mep'] * step4_df['sfc_adjusted_me'] * step4_df['activity_hours']) / 1000000
step4_df['ae_fual_consumption(tonnes)'] = (step4_df['ael'] * step4_df['sfc_adjusted_ae'] * step4_df['activity_hours']) / 1000000
step4_df['ab_fual_consumption(tonnes)'] = (step4_df['abl'] * step4_df['sfc_adjusted_ab'] * step4_df['activity_hours']) / 1000000

step4_df.to_csv('../vessel_movements_dataset_modified.csv', index=False)

